# Elisa real estate, finding a model

Source: https://raw.githubusercontent.com/JulienAlardot/challenge-collecting-data/main/Data/database.csv

Libraries :
OneHotencoding
XGboost
Lasso
Ridge
Random forest
sklearn.preprocessing.LabelEncoder

Expected result: compare the prediction performance of the models, create best simulation and prediction

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
df = pd.read_csv('./data/may.csv', index_col=0)
df.shape

(73510, 23)

In [2]:
df.head(4)

,Url,Source,Locality,Type of property,Subtype of property,Price,Type of sale,Number of rooms,Area,Fully equipped kitchen,...,Terrace Area,Garden,Garden Area,Surface of the land,Surface area of the plot of land,Number of facades,Swimming pool,State of the building,Province,Region
0,https://www.logic-immo.be/fr/vente/appartement...,logic-immo.be,2970,apartment,NaN,319799.0,regular sale,1.0,NaN,0.0,...,11.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,Antwerp,Vlaams
1,https://www.logic-immo.be/fr/vente/appartement...,logic-immo.be,2970,apartment,NaN,291999.0,regular sale,1.0,NaN,0.0,...,6.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,Antwerp,Vlaams
2,https://www.logic-immo.be/fr/vente/appartement...,logic-immo.be,2970,apartment,NaN,764999.0,regular sale,2.0,153.0,0.0,...,62.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,Antwerp,Vlaams
3,https://www.logic-immo.be/fr/vente/appartement...,logic-immo.be,2970,apartment,NaN,660264.0,regular sale,3.0,NaN,0.0,...,160.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,Antwerp,Vlaams


## Data cleaning

 - Existence of duplicated columns ?
 - Get rid of title column ? not needed -> np.array
 - Remove URL column which is there for debugging purpose
 - Remove columns with missing values with threshold more than 80% Nan or equivalent
 - Check if the type of data is homogenous per column if not clean, check unique
 - Check abnomalies and outliers
 
## Data engineering

- Digitalisation of the concrete variables (one, dummies,.. )
- Hyperparameters selection libraries
 

In [3]:
dummies = pd.get_dummies(df.Province)
merged = pd.concat([df, dummies], axis = 'columns')
df = merged.drop(['Province'], axis = 'columns')

In [4]:
# get rid of unuseful data for ML
df.drop(['Url', 'Source', 'Locality', 'Region'], axis=1, inplace = True)

In [5]:
# df.duplicated().any() 
df.drop_duplicates(keep = 'first', inplace=True)
df.shape

(72336, 29)

In [6]:
df.head()

,Type of property,Subtype of property,Price,Type of sale,Number of rooms,Area,Fully equipped kitchen,Furnished,Open fire,Terrace,...,Brabant Wallon,Brussels,Hainaut,Limburg,Liège,Luxembourg,Namur,Oost-Vlanderen,Vlaams-Brabant,West-Vlanderen
0,apartment,NaN,319799.0,regular sale,1.0,NaN,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1,apartment,NaN,291999.0,regular sale,1.0,NaN,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
2,apartment,NaN,764999.0,regular sale,2.0,153.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3,apartment,NaN,660264.0,regular sale,3.0,NaN,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,apartment,NaN,294999.0,regular sale,2.0,80.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0


In [7]:
dummies = pd.get_dummies(df['State of the building'])
merged = pd.concat([df, dummies], axis = 'columns')
df = merged.drop(['State of the building'], axis = 'columns')

In [8]:
# get rid of columns with less than 20% data
df.dropna(thresh=len(df)*0.8, axis='columns', inplace = True)

In [9]:
dummies = pd.get_dummies(df['Type of property'])
merged = pd.concat([df, dummies], axis = 'columns')
df = merged.drop(['Type of property'], axis = 'columns')

In [10]:
# get rid of outliers
# df.sort_values(by='Price', ascending = False).head(4)
df = df[(df.Price >= 40000) & (df.Price <= 10000000)] 
df.shape

(71386, 25)

In [11]:
# examine data of remaining columns 
for columns in df: 
    if(len(df[columns].unique()) < 400): 
        print(f"\n---- {columns} ----\n")
        print(df[columns].unique())


---- Type of sale ----

['regular sale' 'public sale']

---- Number of rooms ----

[  1.   2.   3.  18.   4.  nan   5.   6.  10.   7. 125.  14.   9.  32.
  12.   8.  16.  19.  35.  11.  15.  13.  50.  20.  25.  24.  21.  30.
   0.  47.  66.  22.  36.  40.  41.  34.  17.  28.  46.  63. 165.  27.
  99.  70.  37.  80.  39.  38.  90.  33.  23.]

---- Fully equipped kitchen ----

[0. 1.]

---- Open fire ----

[0. 1.]

---- Terrace ----

[1. 0.]

---- Garden ----

[0. 1.]

---- Swimming pool ----

[0. 1.]

---- Antwerp ----

[1 0]

---- Brabant Wallon ----

[0 1]

---- Brussels ----

[0 1]

---- Hainaut ----

[0 1]

---- Limburg ----

[0 1]

---- Liège ----

[0 1]

---- Luxembourg ----

[0 1]

---- Namur ----

[0 1]

---- Oost-Vlanderen ----

[0 1]

---- Vlaams-Brabant ----

[0 1]

---- West-Vlanderen ----

[0 1]

---- good ----

[0 1]

---- new ----

[0 1]

---- to renovate ----

[0 1]

---- apartment ----

[1 0]

---- house ----

[0 1]


In [12]:
# keep regular sales and drop public sales
# df.groupby([df['Type of sale'] == 'public sale']).count()
df = df[df['Type of sale'] == 'regular sale']
df.shape

(70387, 25)

In [13]:
# clean number of rooms 
# df.groupby(df['Number of rooms'] ).sum()
# df.groupby(df['Number of rooms'] > 9).count()
df =df[df['Number of rooms'] != 0] # more than 1250
df =df[df['Number of rooms'] < 9] # less than 300
df.shape

(67792, 25)

In [14]:
# clea n area
df =df[df['Area'] > 20]
df =df[df['Area'] < 1500]
df.sort_values(by = 'Area')
df.shape

(58377, 25)

In [15]:
# data engineering - is null 
df.isnull().any()


Price                     False
Type of sale              False
Number of rooms           False
Area                      False
Fully equipped kitchen    False
Open fire                 False
Terrace                   False
Garden                    False
Swimming pool             False
Antwerp                   False
Brabant Wallon            False
Brussels                  False
Hainaut                   False
Limburg                   False
Liège                     False
Luxembourg                False
Namur                     False
Oost-Vlanderen            False
Vlaams-Brabant            False
West-Vlanderen            False
good                      False
new                       False
to renovate               False
apartment                 False
house                     False
dtype: bool

In [16]:
# Get rid of nan, delete rows
df = df.dropna()
df.isnull().any()
df.shape

(58377, 25)

In [17]:
df.groupby(df['Number of rooms'] < 1).count()
df.shape

(58377, 25)

In [18]:
# get rid of unuseful data for ML
df.drop(['Type of sale'], axis=1, inplace = True)

In [19]:
df

,Price,Number of rooms,Area,Fully equipped kitchen,Open fire,Terrace,Garden,Swimming pool,Antwerp,Brabant Wallon,...,Luxembourg,Namur,Oost-Vlanderen,Vlaams-Brabant,West-Vlanderen,good,new,to renovate,apartment,house
2,764999.0,2.0,153.0,0.0,0.0,1.0,0.0,0.0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,294999.0,2.0,80.0,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,1,0,0,0,0,1,0
6,233999.0,2.0,90.0,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,0,1,0,0,0,1,0
7,329899.0,1.0,87.0,0.0,0.0,1.0,0.0,0.0,1,0,...,0,0,0,0,0,0,0,0,1,0
9,359899.0,1.0,95.0,0.0,0.0,1.0,0.0,0.0,1,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73496,334900.0,3.0,165.0,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,1
73497,340500.0,3.0,167.0,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,1
73500,307242.0,3.0,150.0,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,0,1,0,0,0,0,1
73501,315000.0,3.0,150.0,0.0,0.0,1.0,0.0,0.0,0,0,...,0,0,1,0,0,1,0,0,0,1


## Create train, test
WARNING 
- never take outliers before the train set, they are part of the tests values
- only clean wrong data, not possible data, error data, etc..
- clean outliers in the train data and monitor over/under fitting aftwards

In [20]:
# create the matrix of x and the y target
X = df.iloc[:,1:] 
y = df.iloc[:,:1] 

print(X.shape) 
print(y.shape)


(58377, 23)
(58377, 1)


In [21]:
# split into train and test data
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=40, stratify = y) # 0,1 class can not be stratified
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=40)
len(X_train)

40863

In [22]:
# TODO measure the accuracy of the train test


## Application of ML

choosing your model : https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html

- multiple regression
- polynomial
- random forest
- xgboost
- ridge
LATER
- libraries cleaning and engineering by themselves


### multiple linear regression
$$
\begin{bmatrix}
y^{(1)}\\
y^{(2)}\\
y^{(3)}\\
... \\
y^{(m)}\\
\end{bmatrix}
=
\begin{bmatrix}
x^{(1)}_1, x^{(1)}_2, 1\\
x^{(2)}_1, x^{(2)}_2, 1\\
x^{(3)}_1, x^{(3)}_2, 1\\
x^{(m)}_1,x^{(m)}_2,  1\\
\end{bmatrix}
.
\begin{bmatrix}
a\\
b\\
c\\
\end{bmatrix}
$$

In [23]:
# multiple linear regression
# with sklearn, no need to add 1 to the matrix
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train,y_train)

LinearRegression()

In [24]:
# performance 0,4037 with number of rooms > 7
regressor.score(X_train, y_train)

0.4849571669642677

In [25]:
# visualizing results


In [26]:
# play with number of iterations


In [27]:
# calculate overfitting and underfitting


### polynomial
$$ X =
\begin{bmatrix}
x^{(1)}_1, x^{(1)2}_2, ..., x^{(m)k}_{n}, 1\\
x^{(2)}_1, x^{(2)2}_2, ..., x^{(m)k}_{n}, 1\\
x^{(3)}_1, x^{(3)2}_2, ..., x^{(m)k}_{n}, 1\\
x^{(m)}_1,x^{(m)k}_2, ..., x^{(m)k}_{n}, 1\\
\end{bmatrix}
$$
is it applicable here ?

In [28]:
# polynomial regression using SciKitLearn


## XGboost

INSTALL package via conda install xgboost

TODO to be used only with 0.5+ score ?


In [29]:
from xgboost import XGBRegressor
my_model = XGBRegressor()
# Add silent=True to avoid printing out updates with each cycle
my_model.fit(X_train, y_train, verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [30]:
# make predictions
predictions = my_model.predict(X_test)

from sklearn.metrics import mean_absolute_error
print("Mean Absolute Error : " + str(mean_absolute_error(predictions, y_test)))
my_model.score(X_test, y_test)

Mean Absolute Error : 103712.3279619608


0.6547313481209388

In [31]:
# finetuning the model
my_model = XGBRegressor(n_estimators=1000)
my_model.fit(X_train, y_train, early_stopping_rounds=50, 
             eval_set=[(X_test, y_test)], verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [32]:
# make predictions
predictions = my_model.predict(X_test)
predictions.shape
# from sklearn.metrics import mean_absolute_error
# print("Mean Absolute Error : " + str(mean_absolute_error(predictions, y_test)))

(17514,)

In [33]:
# mesure performance
my_model.score(X_test, y_test)

0.6568082322490618

## History of performance

#### Options: 

0 < rooms < 9, 

20 < areas < 1500, 

20000 < price < 10 000 000

MultiLinReg = 0,41 ; XGBoost = 0,48   




# Predict with unseen values

